In [1]:
import numpy 
import pandas as pd

import requests
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from base64 import b64encode
import time
import PIconnect as PI
from PIconnect.PIConsts import UpdateMode, BufferMode

import warnings
warnings.filterwarnings("ignore")

import PIconnect as PI
print(list(PI.PIServer.servers.keys()))

ROOT_URL = "https://127.0.0.1/piwebapi"
COMPUTER = "WIN-5MGCA5NMS7S"
DATABASE = "Database1"
USERNAME = f"{COMPUTER}\Administrator"
PASSWORD = "abcd1234.."

def basic_auth(username, password):
    token = b64encode(f"{username}:{password}".encode('utf-8')).decode("ascii")
    return f'Basic {token}'

BASIC_AUTH_TOKEN = basic_auth(USERNAME, PASSWORD)
POSTHEADERS = { 'X-Requested-With': 'True', 'Content-Type': 'application/json', 'Authorization': BASIC_AUTH_TOKEN }

OSIsoft(r) AF SDK Version: 3.0.0.790
['WIN-5MGCA5NMS7S']


In [2]:
# Test Server
response_server = requests.get(f"{ROOT_URL}/", auth=(USERNAME, PASSWORD), verify=False)
response_server.json()

{'Links': {'Self': 'https://127.0.0.1/piwebapi/',
  'AssetServers': 'https://127.0.0.1/piwebapi/assetservers',
  'DataServers': 'https://127.0.0.1/piwebapi/dataservers',
  'Omf': 'https://127.0.0.1/piwebapi/omf',
  'System': 'https://127.0.0.1/piwebapi/system'}}

In [12]:
# Get Database WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/assetdatabases?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
DATABASE_WEBID = rsp_element_raw.json()['WebId']

# Get DATASERVER WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/dataservers", auth=(USERNAME, PASSWORD), verify=False)
DATASERVER_WEBID = rsp_element_raw.json()['Items'][0]['WebId']

# Get By Path Element
elementPath = f"\\\\{COMPUTER}\\{DATABASE}\\Data_Site\\LGS"
rsp_element_raw = requests.get(f"{ROOT_URL}/elements?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
rsp_element_json = rsp_element_raw.json()
ELEMENT_TARGET = rsp_element_json['WebId']

In [3]:
df = pd.read_excel('LGS#1_Overall 2017-2023 (hourly).xlsx', "LGS#1 2023")
df.head()

,TimeStamp,Grid (Aux: 0 & FCE:1),Active Power (MW),Reactive Power (MVAr),Exc Voltage (Volt),Exc Current (A),V Phase 1 (Volt),V Phase 2 (Volt),V Phase 3 (Volt),Current Phase 1 (A),...,LGB Metal Temp 2 (Celcius),LGB Oil Temp (Celcius),Gen air Cooler Air Otlet Temp (Celcius),Gen air Cooler Water Inlet Temp (Celcius),Gen air Cooler Water Otlet Temp (Celcius),Gen Cooler Water Flow (LPM),UGB Cooling Water Flow (LPM),LGB Cooling Water Flow (LPM),Lower Canal Level (msAL),TailRace Level (msAL)
0,2023-01-01 00:00:01,0.0,37.61,19.07,123.82,702.02,10.77,10.73,10.84,0.0,...,53.0,70.0,55.14,61.01,59.31,3795.33,32.42,945.70,308.1,165.96
1,2023-01-01 01:00:01,0.0,37.51,19.04,123.86,704.99,10.77,10.73,10.83,0.0,...,53.0,70.0,55.26,61.63,60.00,3797.60,32.42,946.46,308.1,165.97
2,2023-01-01 02:00:01,0.0,37.34,19.00,123.34,702.22,10.76,10.72,10.83,0.0,...,53.0,70.0,55.39,62.00,60.00,3793.92,32.42,944.96,308.1,165.98
3,2023-01-01 03:00:02,0.0,37.17,18.97,124.49,703.04,10.76,10.72,10.82,0.0,...,53.0,70.0,55.51,62.00,60.00,3798.31,32.42,947.38,308.1,165.99
4,2023-01-01 04:00:02,0.0,37.01,18.93,121.03,702.86,10.75,10.72,10.82,0.0,...,53.0,70.0,55.64,62.00,60.00,3806.77,32.42,948.19,308.1,166.00


In [4]:
timestamp_df = pd.to_datetime(df.iloc[:,0])
grid_df = df.iloc[:,1] 
insert_df = df.iloc[:,2:].fillna(-5)

timestamp_df

0      2023-01-01 00:00:01
1      2023-01-01 01:00:01
2      2023-01-01 02:00:01
3      2023-01-01 03:00:02
4      2023-01-01 04:00:02
               ...        
7291   2023-10-31 19:59:57
7292   2023-10-31 20:59:57
7293   2023-10-31 21:59:58
7294   2023-10-31 22:59:58
7295   2023-10-31 23:59:59
Name: TimeStamp, Length: 7296, dtype: datetime64[ns]

In [ ]:
for series_name, series in insert_df.items():
    name_data = series_name.replace(")", "").split(" (")[0]
    uom = series_name.replace(")", "").split("(")[1]
    series = series.astype('float32')
    print(series_name)
    
    rsp_do_point_raw = requests.post(f"{ROOT_URL}/dataservers/{DATASERVER_WEBID}/points", headers=POSTHEADERS, data=json.dumps({
        "Name": "LGS_" + name_data,
        "Descriptor": "LGS " + name_data,
        "PointClass": "classic",
        "PointType": "float32",
        "EngineeringUnits": "",
        "Step": False,
        "Future": False,
        "DisplayDigits": -5
    }), verify=False)
    #print(rsp_do_point_raw)

    rsp_do_raw = requests.post(f"{ROOT_URL}/elements/{ELEMENT_TARGET}/attributes", headers=POSTHEADERS, data=json.dumps({
        "Name": name_data,
        "Description": name_data,
        "Type": "Double",
        "TypeQualifier": "",
        "DefaultUnitsName": '',
        "DisplayDigits": -5,
        "DataReferencePlugIn": "PI Point",
        "ConfigString": "LGS_" + name_data,
        "IsConfigurationItem": False,
        "IsHidden": False,
        "IsManualDataEntry": False
    }), verify=False)
    #print(rsp_do_raw)
    time.sleep(1)
    

In [10]:
for series_name, series in insert_df.items():
    name_data = series_name.replace(")", "").split(" (")[0]
    uom = series_name.replace(")", "").split("(")[1]
    series = series.astype('float32')
    current_point_name = "LGS_" + name_data
    print(series_name)

    with PI.PIServer(server=COMPUTER) as server:
        point = server.search(current_point_name)[0]
        for index, value in series.items():
            point.update_value(
                    value,
                    timestamp_df[index].to_pydatetime(),
                    UpdateMode.NO_REPLACE,
                    BufferMode.DO_NOT_BUFFER,
                )

            # time.sleep(50/1000)
            # print(index / len(series))

Active Power (MW)


ArgumentException: The supplied DateTime represents an invalid time.  For example, when the clock is adjusted forward, any time in the period that is skipped is invalid.
Parameter name: dateTime
   at System.TimeZoneInfo.ConvertTime(DateTime dateTime, TimeZoneInfo sourceTimeZone, TimeZoneInfo destinationTimeZone, TimeZoneInfoOptions flags, CachedData cachedData)
   at OSIsoft.AF.Time.AFTimeZone.ConvertTimeToUtc(DateTime dateTime, TimeZoneInfo sourceTimeZone)
   at OSIsoft.AF.Time.AFTimeZone.ToUniversalTime(AFTimeZone source, DateTime time)
   at OSIsoft.AF.Time.AFTime.LocalTimeToUtcTime(DateTime localTime, AFTimeZone timeZone, Boolean clamp)
   at OSIsoft.AF.Time.AFTime.TryParseImpl(String input, Nullable`1 referenceTime, AFTimeZone timeZone, IFormatProvider provider, Boolean throwErrors, DateTime& result)
   at OSIsoft.AF.Time.AFTime.TryParseRetryImpl(String input, Nullable`1 referenceTime, IFormatProvider provider, Boolean toUtcTime, Boolean throwErrors, DateTime& result)
   

In [8]:
timestamp_df[index-1]

Timestamp('2023-03-12 01:13:50')

In [9]:
timestamp_df[index]

Timestamp('2023-03-12 02:13:50')

In [11]:
timestamp_df[index+1]

Timestamp('2023-03-12 03:13:51')

In [7]:
timestamp_df[index-1].to_pydatetime()

datetime.datetime(2023, 3, 12, 1, 13, 50)

In [6]:
timestamp_df[index].to_pydatetime()

datetime.datetime(2023, 3, 12, 2, 13, 50)

In [56]:
timestamp_df[index-2].to_pydatetime()

datetime.datetime(2023, 3, 12, 0, 13, 49)

In [57]:
timestamp_df[index+1].to_pydatetime()

datetime.datetime(2023, 3, 12, 3, 13, 51)

'TailRace Level'

In [51]:
point

[PIPoint(LGS_TailRace Level, LGS TailRace Level; Current Value: Pt Created )]

In [43]:
uom

'msAL'

In [42]:
rsp_do_raw.headers

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Security-Policy': "default-src 'self'; object-src 'none'", 'X-Content-Type-Options': 'nosniff', 'Strict-Transport-Security': 'max-age=86400', 'Date': 'Thu, 02 May 2024 13:27:58 GMT'}

In [37]:
rsp_do_point_raw

<Response [409]>

In [35]:
rsp_do_raw

<Response [400]>

In [30]:
series_name

'Modbus MVAr'

In [24]:
pd.to_datetime(df.iloc[:,0])[0]

datetime.datetime(2023, 1, 1, 0, 0, 1)

In [11]:
with PI.PIServer(server=COMPUTER) as server:
    print(server.server_name)

WIN-5MGCA5NMS7S


['WIN-5MGCA5NMS7S']
